<a href="https://colab.research.google.com/github/ShahdBahaaa/text_summarizer/blob/main/flan_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets sentencepiece accelerate rouge-score

  Preparing metadata (setup.py) ... done


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments
)
from rouge_score import rouge_scorer

In [3]:
# 1. Load dataset & model
dataset = load_dataset("cnn_dailymail", "3.0.0", trust_remote_code=True)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'cnn_dailymail' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'cnn_dailymail' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this s

README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
model_name = "google/flan-t5-base"          # or "large" / "xl" if you have GPU memory
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [17]:
# 2. Preprocessing — NO fixed "summarize: " prefix
def preprocess(batch):
    # Clean version — model learns to summarize from raw article
    inputs = [f"summarize: {doc}" for doc in batch["article"]]

    # Alternative (if you want slightly more instruction signal during training):
    # inputs = [f"Summarize the article:\n{doc}" for doc in batch["article"]]

    targets = batch["highlights"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
    )

    labels = tokenizer(
        targets,
        max_length=128,
        truncation=True,
        padding="max_length",
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [27]:
# Small subset for quick experiments
small_train = dataset["train"].shuffle(seed=42).select(range(1000))
small_val   = dataset["validation"].shuffle(seed=42).select(range(100))

tokenized_train = small_train.map(preprocess, batched=True, remove_columns=small_train.column_names)
tokenized_val   = small_val.map(preprocess, batched=True, remove_columns=small_val.column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [28]:
# 3. Training arguments (almost same as yours)
training_args = TrainingArguments(
    output_dir="./flan_t5_flex_summarizer",
    eval_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="none",
    load_best_model_at_end=True,         # small but useful addition
    metric_for_best_model="rougeL",      # requires compute_metrics (see below)
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [29]:
# 4. Optional: better eval metric during training
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[l] if not isinstance(l, list) else l for l in labels]  # make sure list of lists
    decoded_labels = tokenizer.batch_decode(
        [[tok if tok != -100 else tokenizer.pad_token_id for tok in lbl] for lbl in labels],
        skip_special_tokens=True
    )

    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    result = {}
    for pred, ref in zip(decoded_preds, decoded_labels):
        score = scorer.score(ref, pred)
        for k, v in score.items():
            result.setdefault(k, []).append(v.fmeasure)

    return {k: sum(v)/len(v) for k,v in result.items()}

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2812472608.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [31]:
# 5. Train
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=375, training_loss=0.0, metrics={'train_runtime': 367.1883, 'train_samples_per_second': 8.17, 'train_steps_per_second': 1.021, 'total_flos': 2054272057344000.0, 'train_loss': 0.0, 'epoch': 3.0})

In [32]:
def summarize(
    text: str,
    instruction: str = "Summarize the following article in 3-5 sentences:",
    max_length: int = 140,
    min_length: int = 40,
) -> str:
    prompt = f"{instruction}\n\n{text}"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_length=max_length,
        min_length=min_length,
        num_beams=5,
        length_penalty=1.0,
        no_repeat_ngram_size=3,
        early_stopping=True,
        do_sample=False,           # greedy + beam usually best for summarization
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

In [33]:
# 7. Quick test + ROUGE
sample = dataset["test"][0]["article"]
reference = dataset["test"][0]["highlights"]

generated = summarize(sample)  # default instruction

scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
scores = scorer.score(reference, generated)

In [34]:
print("Generated:\n", generated)
print("\nReference:\n", reference)
print("\nROUGE:", {k: round(v.fmeasure, 4) for k,v in scores.items()})

Generated:
 Palestinians' move to become 123rd member of the International Criminal Tribunal for the Palestinian Authority, paving the way for possible war crimes investigations against Israelis in the occupied West Bank.

Reference:
 Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

ROUGE: {'rouge1': 0.4062, 'rouge2': 0.129, 'rougeL': 0.2812}


In [35]:
# Save for Streamlit
model.save_pretrained("./flan_t5_flex_finetuned")
tokenizer.save_pretrained("./flan_t5_flex_finetuned")

('./flan_t5_flex_finetuned/tokenizer_config.json',
 './flan_t5_flex_finetuned/special_tokens_map.json',
 './flan_t5_flex_finetuned/spiece.model',
 './flan_t5_flex_finetuned/added_tokens.json')